# Librerías

In [2]:
import requests #mandar un requerimiento a la API
import json #poder estructurar nuestros datos
import pandas as pd #para poder visualizar los datos en una estructura tabular

import warnings
warnings.filterwarnings("ignore")

# Lo que se vió en clases

In [ ]:
#Solo obtuvimos los resultados de la carrera # 22 y lo hicimos de la siguiente forma:
url = 'http://ergast.com/api/f1/2022/22/results.json'
response = requests.get(url)
response.content
content = json.loads(response.content)
pd.DataFrame(content['MRData']['RaceTable']['Races'][0]['Results'])


#Ahora queremos la obtencion de todas las carreras, y hacemos lo siguiente
for race in list(range(1,23)):
    url = 'http://ergast.com/api/f1/2022/{}/results.json'
    response = requests.get(url.format(race))
    content = json.loads(response.content)
    
#vemos el ultimo registro
pd.DataFrame(content['MRData']['RaceTable']['Races'][0]['Results'])

#pero lo que queremos es ver todos los registros, para ello hacemos lo siguiente:
#1) Hacemos un DataFrame
df_results_2022=pd.DataFrame()
#2) 
for race in list(range(1,23)):
    url = 'http://ergast.com/api/f1/2022/{}/results.json'
    response = requests.get(url.format(race))
    content = json.loads(response.content)
    df_results_2022 =df_results_2022.append(pd.DataFrame(content['MRData']['RaceTable']['Races'][0]['Results']))
    
df_results_2022.head()
#Al correr el codigo van a salir warnings, para ello importamos antes lo siguiente:
#import warnings
#warnings.filterwarnings("ignore")

In [ ]:
#Cuántos puntos obtuvo cada piloto en esa temporada.
#queremos sumar los puntos
print(df_results_2022.groupby(['position']).agg({'points':'sum'}))
#pero vemos que arroja cosas raras, ¿por qué?, para dar respuesta a ello vamos a ver que tipo es 'points'
print(type(df_results_2022.points.values[0]))
#entonces hay que convertir toda la columna a entero:
df_results_2022.points.astype(int)
#verificamos que en efecto sean enteros
print(type(df_results_2022.points.astype(int).values[0]))
df_results_2022['points']=df_results_2022.points.astype(int)
df_results_2022.head(3)
df_results_2022.groupby(['position']).agg({'points':'sum'}).sort_values(by='points', ascending =False)

In [ ]:
#Vamos a extraer solamente el nombre del conductor 
df_results_2022['Driver']=df_results_2022['Driver'].apply(lambda x:x['driverId'])
df_results_2022.head(3)

In [ ]:
df_results_2022.groupby(['Driver']).agg({'points': 'sum'}).sort_values(by = 'points', ascending =False)

In [ ]:
#Puntos por constuctor
df_results_2022['Constructor'] = df_results_2022['Constructor'].apply(lambda x:x['constructorId'])
df_results_2022.groupby(['Constructor']).agg({'points':'sum'}).sort_values(by = 'points', ascending=True)

# ANÁLISIS DE LA F1 

# 1) Resultados de las carreras desde el 2010 hasta el 2023

In [3]:
races = {'season':[],
         'round':[],
        'raceName':[],
        'circuitId':[],
        'lat':[],
        'long':[],
        'country':[],
        'date':[],
        'url':[]}

#Vamos a traernos todas las carreras del año 2010 hasta el año 2022, para ello en el url las "{}" significa
# que vamos a extraer los resultados de una carrera específica con el año con el que indique el 'for'
for year in list(range(2010,2024)):
    url = 'http://ergast.com/api/f1/{}.json'
    #Va a obtener la información del link con el año que le indiquemos con '.formar(year)'
    response = requests.get(url.format(year)) 
    #Contenido del url en formato json
    content = json.loads(response.content)
    
    #content['MRData']['RaceTable']['Races'] es una lista, la cual contiene 19 diccionario, 
    #esto quiere decir que por cada año vamos a obtener los datos definidos en el diccionario 'races' 19 veces.
    
    for item in content['MRData']['RaceTable']['Races']:
        
        races['season'].append(int(item['season']))
        races['round'].append(int(item['round']))
        races['raceName'].append(item['raceName'])
        races['circuitId'].append(item['Circuit']['circuitId'])
        races['lat'].append(float(item['Circuit']['Location']['lat']))
        races['long'].append(float(item['Circuit']['Location']['long']))
        races['country'].append(item['Circuit']['Location']['country'])
        races['date'].append(item['date'])
        races['url'].append(item['url'])


dfraces=pd.DataFrame(races)
dfraces['date']= dfraces['date'].astype('datetime64[ns]')
pd.DataFrame(dfraces)

,season,round,raceName,circuitId,lat,long,country,date,url
0,2010,1,Bahrain Grand Prix,bahrain,26.03250,50.51060,Bahrain,2010-03-14,http://en.wikipedia.org/wiki/2010_Bahrain_Gran...
1,2010,2,Australian Grand Prix,albert_park,-37.84970,144.96800,Australia,2010-03-28,http://en.wikipedia.org/wiki/2010_Australian_G...
2,2010,3,Malaysian Grand Prix,sepang,2.76083,101.73800,Malaysia,2010-04-04,http://en.wikipedia.org/wiki/2010_Malaysian_Gr...
3,2010,4,Chinese Grand Prix,shanghai,31.33890,121.22000,China,2010-04-18,http://en.wikipedia.org/wiki/2010_Chinese_Gran...
4,2010,5,Spanish Grand Prix,catalunya,41.57000,2.26111,Spain,2010-05-09,http://en.wikipedia.org/wiki/2010_Spanish_Gran...
...,...,...,...,...,...,...,...,...,...
277,2023,19,United States Grand Prix,americas,30.13280,-97.64110,USA,2023-10-22,https://en.wikipedia.org/wiki/2023_United_Stat...
278,2023,20,Mexico City Grand Prix,rodriguez,19.40420,-99.09070,Mexico,2023-10-29,https://en.wikipedia.org/wiki/2023_Mexico_City...
279,2023,21,São Paulo Grand Prix,interlagos,-23.70360,-46.69970,Brazil,2023-11-05,https://en.wikipedia.org/wiki/2023_S%C3%A3o_Pa...
280,2023,22,Las Vegas Grand Prix,vegas,36.11470,-115.17300,United States,2023-11-19,https://en.wikipedia.org/wiki/2023_Las_Vegas_G...


In [4]:
pd.DataFrame(dfraces).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   season     282 non-null    int64         
 1   round      282 non-null    int64         
 2   raceName   282 non-null    object        
 3   circuitId  282 non-null    object        
 4   lat        282 non-null    float64       
 5   long       282 non-null    float64       
 6   country    282 non-null    object        
 7   date       282 non-null    datetime64[ns]
 8   url        282 non-null    object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 20.0+ KB


In [ ]:
pd.DataFrame(dfraces).to_csv('races.csv', index=False)


# 2) Seasons

Las temporadas actualmente admitidad por la API solo van del año 1950 hasta 1979.

In [ ]:
url = 'http://ergast.com/api/f1/seasons.json'
response = requests.get(url)
content = json.loads(response.content)
content['MRData']['SeasonTable']['Seasons']

season = { 'season':[], 
           'url':[]
         }

for item in content['MRData']['SeasonTable']['Seasons']:
    season['season'].append(int(item['season']))
    season['url'].append(item['url'])
    
pd.DataFrame(season)

In [ ]:
pd.DataFrame(season).info()

In [ ]:
pd.DataFrame(season).to_csv('season.csv', index=False)

# 3) Drivers desde el 2010 hasta el 2023

In [ ]:
drivers = {'season':[],
           'driverId':[],
           'code':[],
           'givenName':[],
           'familyName':[],
           'dateOfBirth':[],
           'nationality':[],
           'url':[],
          }

for year in list(range(2010,2023)):
    url='http://ergast.com/api/f1/{}/drivers.json'
    response = requests.get(url.format(year))
    content = json.loads(response.content);content

    for item in content['MRData']['DriverTable']['Drivers']:
        drivers['season'].append(int(year))
        drivers['driverId'].append(item['driverId'])
        drivers['code'].append(item['code'])
        drivers['url'].append(item['url'])
        drivers['givenName'].append(item['givenName'])
        drivers['familyName'].append(item['familyName'])
        drivers['dateOfBirth'].append(item['dateOfBirth'])
        drivers['nationality'].append(item['nationality'])

        
dfdrivers=pd.DataFrame(drivers)
dfdrivers['dateOfBirth']= dfdrivers['dateOfBirth'].astype('datetime64[ns]')
pd.DataFrame(dfdrivers)

In [ ]:
 pd.DataFrame(dfdrivers).info()

In [ ]:
pd.DataFrame(dfdrivers).to_csv('drivers.csv', index=False)

# 4) Constructors desde el 2010 hasta el 2023

In [ ]:
constructors = {'season':[],
           'constructorId':[],
           'name':[],
           'nationality':[],
           'url':[]
          }

for year in list(range(2010,2023)):
    url='http://ergast.com/api/f1/{}/constructors.json'
    response = requests.get(url.format(year))
    content = json.loads(response.content)
    
    for item in content['MRData']['ConstructorTable']['Constructors']:
        constructors['season'].append(int(year))
        constructors['constructorId'].append(item['constructorId'])
        constructors['name'].append(item['name'])
        constructors['nationality'].append(item['nationality'])
        constructors['url'].append(item['url'])
    
pd.DataFrame(constructors)


In [ ]:
pd.DataFrame(constructors).info()

In [ ]:
pd.DataFrame(constructors).to_csv('constructors.csv', index=False)

# 5) Circuits desde el 2010 hasta el 2023

In [ ]:
circuit = {'season':[],
           'circuitId':[],
           'circuitName':[],
           'lat':[],
           'long':[],
           'locality':[],
           'country':[],
           'url':[]
          }

for year in list(range(2010,2023)):
    url='http://ergast.com/api/f1/{}/circuits.json'
    response = requests.get(url.format(year))
    content = json.loads(response.content)
    
    for item in content['MRData']['CircuitTable']['Circuits']:
        circuit['season'].append(int(year))
        circuit['circuitId'].append(item['circuitId'])
        circuit['circuitName'].append(item['circuitName'])
        circuit['lat'].append(float(item['Location']['lat']))
        circuit['long'].append(float(item['Location']['long']))
        circuit['locality'].append(item['Location']['locality'])
        circuit['country'].append(item['Location']['country'])
        circuit['url'].append(item['url'])
    
pd.DataFrame(circuit)

In [ ]:
pd.DataFrame(circuit).info()

In [ ]:
pd.DataFrame(circuit).to_csv('circuit.csv', index=False)

# 6) Qualifying

In [ ]:
qualifying = {'season':[],
              'number':[],
              'position':[],
              'driverId':[],
              'constructorId':[],
              'Q1':[],
              'Q2':[],
              'Q3':[]
          }

for year in list(range(2010,2023)):
    url = 'http://ergast.com/api/f1/{}/qualifying.json'
    response = requests.get(url.format(year))
    content = json.loads(response.content)
    
    for item in content['MRData']['RaceTable']['Races'][0]['QualifyingResults']:
        qualifying['season'].append(year)
        qualifying['number'].append(int(item['number']))
        qualifying['position'].append(int(item['position']))
        qualifying['driverId'].append(item['Driver']['driverId'])
        qualifying['constructorId'].append(item['Constructor']['constructorId'])
        try:
            qualifying['Q1'].append(item['Q1'])
        except:
            qualifying['Q1'].append(None) 
        try:
            qualifying['Q2'].append(item['Q2'])
        except:
            qualifying['Q2'].append(None)
        try:
            qualifying['Q3'].append(item['Q3'])
        except:
            qualifying['Q3'].append(None) 

    for item in content['MRData']['RaceTable']['Races'][1]['QualifyingResults']:
        qualifying['season'].append(year)
        qualifying['number'].append(int(item['number']))
        qualifying['position'].append(int(item['position']))
        qualifying['driverId'].append(item['Driver']['driverId'])
        qualifying['constructorId'].append(item['Constructor']['constructorId'])
        try:
            qualifying['Q1'].append(item['Q1'])
        except:
            qualifying['Q1'].append(None) 
        try:
            qualifying['Q2'].append(item['Q2'])
        except:
            qualifying['Q2'].append(None)
        try:
            qualifying['Q3'].append(item['Q3'])
        except:
            qualifying['Q3'].append(None)       
pd.DataFrame(qualifying)

In [ ]:
pd.DataFrame(qualifying).info()

In [ ]:
pd.DataFrame(qualifying).to_csv('qualifying.csv', index=False)


# 7) SprintResults

In [2]:
SprintResults = {'raceName':[],
              'driverId':[],
              'constructorId':[],
              'number':[],
              'grid':[],
              'position':[],
              'positionText':[],
              'points':[],
              'laps':[],
              'time':[],
              'millis':[],
              'status':[],
              'fastestlap':[],
              'fastestlaptime':[]
          }

for year in list(range(2021,2023)):
    url = 'http://ergast.com/api/f1/{}/sprint.json'
    response = requests.get(url.format(year))
    content = json.loads(response.content)

    for i in list(range(0, len(content['MRData']['RaceTable']['Races']))):
        for item in content['MRData']['RaceTable']['Races'][i]['SprintResults']:
            SprintResults['raceName'].append(content['MRData']['RaceTable']['Races'][i]['raceName'])
            SprintResults['driverId'].append(item['Driver']['driverId'])
            SprintResults['constructorId'].append(item['Constructor']['constructorId'])
            SprintResults['number'].append(int(item['number']))
            SprintResults['grid'].append(int(item['grid']))
            SprintResults['position'].append(int(item['position']))
            SprintResults['positionText'].append(item['positionText'])
            SprintResults['points'].append(int(item['points']))
            SprintResults['laps'].append(int(item['laps']))
            SprintResults['status'].append(item['status'])
            try:
                SprintResults['time'].append(item['Time']['time'])
            except:
                SprintResults['time'].append(None)
            try:
                SprintResults['millis'].append(int(item['Time']['millis']))
            except:
                SprintResults['millis'].append(None)
            try:
                SprintResults['fastestlap'].append(item['FastestLap']['lap'])
            except:
                SprintResults['fastestlap'].append(None)
            try:
                SprintResults['fastestlaptime'].append(item['FastestLap']['Time']['time'])
            except:
                SprintResults['fastestlaptime'].append(None)
                          


pd.DataFrame(SprintResults)

,raceName,driverId,constructorId,number,grid,position,positionText,points,laps,time,millis,status,fastestlap,fastestlaptime
0,British Grand Prix,max_verstappen,red_bull,33,2,1,1,3,17,25:38.426,1538426.0,Finished,14,1:30.013
1,British Grand Prix,hamilton,mercedes,44,1,2,2,2,17,+1.430,1539856.0,Finished,17,1:29.937
2,British Grand Prix,bottas,mercedes,77,3,3,3,1,17,+7.502,1545928.0,Finished,17,1:29.958
3,British Grand Prix,leclerc,ferrari,16,4,4,4,0,17,+11.278,1549704.0,Finished,16,1:30.163
4,British Grand Prix,norris,mclaren,4,6,5,5,0,17,+24.111,1562537.0,Finished,16,1:30.566
5,British Grand Prix,ricciardo,mclaren,3,7,6,6,0,17,+30.959,1569385.0,Finished,17,1:30.640
6,British Grand Prix,alonso,alpine,14,11,7,7,0,17,+43.527,1581953.0,Finished,17,1:31.773
7,British Grand Prix,vettel,aston_martin,5,10,8,8,0,17,+44.439,1582865.0,Finished,17,1:31.687
8,British Grand Prix,russell,williams,63,8,9,9,0,17,+46.652,1585078.0,Finished,17,1:32.208
9,British Grand Prix,ocon,alpine,31,13,10,10,0,17,+47.395,1585821.0,Finished,16,1:32.183


In [3]:
pd.DataFrame(SprintResults).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   raceName        60 non-null     object 
 1   driverId        60 non-null     object 
 2   constructorId   60 non-null     object 
 3   number          60 non-null     int64  
 4   grid            60 non-null     int64  
 5   position        60 non-null     int64  
 6   positionText    60 non-null     object 
 7   points          60 non-null     int64  
 8   laps            60 non-null     int64  
 9   time            58 non-null     object 
 10  millis          58 non-null     float64
 11  status          60 non-null     object 
 12  fastestlap      59 non-null     object 
 13  fastestlaptime  59 non-null     object 
dtypes: float64(1), int64(5), object(8)
memory usage: 6.7+ KB


# 8) Status 

In [4]:
status = { 'season':[],
          'statusId':[],
          'status':[]
         }

for year in list(range(2010,2023)):
    url='http://ergast.com/api/f1/{}/status.json'
    response = requests.get(url.format(year))
    content = json.loads(response.content)
    for i in list(range(0, len(content['MRData']['StatusTable']['Status']))):
            status['season'].append(int(year))
            status['statusId'].append(content['MRData']['StatusTable']['Status'][i]['statusId'])
            status['status'].append(content['MRData']['StatusTable']['Status'][i]['status'])

pd.DataFrame(status)

,season,statusId,status
0,2010,1,Finished
1,2010,3,Accident
2,2010,4,Collision
3,2010,5,Engine
4,2010,6,Gearbox
...,...,...,...
356,2022,103,Water pump
357,2022,130,Collision damage
358,2022,131,Power Unit
359,2022,140,Undertray


In [5]:
pd.DataFrame(status).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361 entries, 0 to 360
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   season    361 non-null    int64 
 1   statusId  361 non-null    object
 2   status    361 non-null    object
dtypes: int64(1), object(2)
memory usage: 8.6+ KB


# 9) Results

In [6]:
Results = {'raceName':[],
              'driverId':[],
              'constructorId':[],
              'number':[],
              'grid':[],
              'position':[],
              'positionText':[],
              'points':[],
              'laps':[],
              'time':[],
              'millis':[],
              'fastestlap':[],
              'rank':[],
              'fastestlaptime':[],
              'fastestlapspeed':[],
              'status':[],
          }

for year in list(range(2010,2023)):
    url = 'http://ergast.com/api/f1/{}/results.json'
    response = requests.get(url.format(year))
    content = json.loads(response.content)

    for i in list(range(0, len(content['MRData']['RaceTable']['Races']))):
        for item in content['MRData']['RaceTable']['Races'][i]['Results']:
            Results['raceName'].append(content['MRData']['RaceTable']['Races'][i]['raceName'])
            Results['driverId'].append(item['Driver']['driverId'])
            Results['constructorId'].append(item['Constructor']['constructorId'])
            Results['number'].append(int(item['number']))
            Results['grid'].append(int(item['grid']))
            Results['position'].append(int(item['position']))
            Results['positionText'].append(item['positionText'])
            Results['points'].append(int(item['points']))
            Results['laps'].append(int(item['laps']))
            Results['status'].append(item['status'])
            try:
                Results['time'].append(item['Time']['time'])
            except:
                Results['time'].append(None)
            try:
                Results['millis'].append(int(item['Time']['millis']))
            except:
                Results['millis'].append(None)
            try:
                Results['fastestlap'].append(item['FastestLap']['lap'])
            except:
                Results['fastestlap'].append(None)
            try:
                Results['rank'].append(int(item['FastestLap']['rank']))
            except:
                Results['rank'].append(None)
            try:
                Results['fastestlaptime'].append(item['FastestLap']['Time']['time'])
            except:
                Results['fastestlaptime'].append(None)
            try:
                Results['fastestlapspeed'].append(float(item['FastestLap']['AverageSpeed']['speed']))
            except:
                Results['fastestlapspeed'].append(None)


pd.DataFrame(Results)

,raceName,driverId,constructorId,number,grid,position,positionText,points,laps,time,millis,fastestlap,rank,fastestlaptime,fastestlapspeed,status
0,Bahrain Grand Prix,alonso,ferrari,8,3,1,1,25,49,1:39:20.396,5960396.0,45,1.0,1:58.287,191.706,Finished
1,Bahrain Grand Prix,massa,ferrari,7,2,2,2,18,49,+16.099,5976495.0,38,5.0,1:59.732,189.392,Finished
2,Bahrain Grand Prix,hamilton,mclaren,2,4,3,3,15,49,+23.182,5983578.0,42,4.0,1:59.560,189.665,Finished
3,Bahrain Grand Prix,vettel,red_bull,5,1,4,4,12,49,+38.799,5999195.0,32,12.0,2:00.218,188.627,Finished
4,Bahrain Grand Prix,rosberg,mercedes,4,5,5,5,10,49,+40.213,6000609.0,45,13.0,2:00.236,188.599,Finished
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,Saudi Arabian Grand Prix,ocon,alpine,31,5,6,6,8,50,+56.017,5115310.0,46,9.0,1:33.103,238.729,Finished
386,Saudi Arabian Grand Prix,norris,mclaren,4,11,7,7,6,50,+56.124,5115417.0,46,5.0,1:32.753,239.629,Finished
387,Saudi Arabian Grand Prix,gasly,alphatauri,10,9,8,8,4,50,+1:02.946,5122239.0,42,10.0,1:33.468,237.796,Finished
388,Saudi Arabian Grand Prix,kevin_magnussen,haas,20,10,9,9,2,50,+1:04.308,5123601.0,48,6.0,1:32.779,239.562,Finished


In [7]:
pd.DataFrame(Results).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   raceName         390 non-null    object 
 1   driverId         390 non-null    object 
 2   constructorId    390 non-null    object 
 3   number           390 non-null    int64  
 4   grid             390 non-null    int64  
 5   position         390 non-null    int64  
 6   positionText     390 non-null    object 
 7   points           390 non-null    int64  
 8   laps             390 non-null    int64  
 9   time             236 non-null    object 
 10  millis           236 non-null    float64
 11  fastestlap       373 non-null    object 
 12  rank             373 non-null    float64
 13  fastestlaptime   373 non-null    object 
 14  fastestlapspeed  373 non-null    float64
 15  status           390 non-null    object 
dtypes: float64(3), int64(5), object(8)
memory usage: 48.9+ KB


# 10) PitStops

In [9]:
PitStops = {'raceName': [],
           'driverId':[],
           'stop':[],
           'lap':[],
           'time':[],
           'duration':[]
           }

for year in list(range(2011,2023)):
    url= 'http://ergast.com/api/f1/{}/5/pitstops.json'
    response = requests.get(url.format(year))
    content = json.loads(response.content)
            

    for item in content['MRData']['RaceTable']['Races']:
        for i in list(range(0, len(content['MRData']['RaceTable']['Races'][0]['PitStops']))):
            PitStops['raceName'].append(item['raceName'])
            PitStops['driverId'].append(item['PitStops'][i]['driverId'])
            PitStops['lap'].append(int(item['PitStops'][i]['lap']))
            PitStops['stop'].append(int(item['PitStops'][i]['stop']))
            PitStops['time'].append(item['PitStops'][i]['time'])
            PitStops['duration'].append(item['PitStops'][i]['duration'])
        

pd.DataFrame(PitStops)

,raceName,driverId,stop,lap,time,duration
0,Spanish Grand Prix,kobayashi,1,1,14:05:11,24.871
1,Spanish Grand Prix,perez,1,7,14:14:14,23.592
2,Spanish Grand Prix,maldonado,1,8,14:15:48,22.689
3,Spanish Grand Prix,vettel,1,9,14:16:54,20.253
4,Spanish Grand Prix,buemi,1,9,14:17:17,20.818
...,...,...,...,...,...,...
330,Miami Grand Prix,vettel,1,41,16:39:42,19.861
331,Miami Grand Prix,ricciardo,2,41,16:39:49,19.280
332,Miami Grand Prix,tsunoda,2,41,16:40:43,19.618
333,Miami Grand Prix,kevin_magnussen,2,42,16:41:34,20.673


In [10]:
pd.DataFrame(PitStops).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   raceName  335 non-null    object
 1   driverId  335 non-null    object
 2   stop      335 non-null    int64 
 3   lap       335 non-null    int64 
 4   time      335 non-null    object
 5   duration  335 non-null    object
dtypes: int64(2), object(4)
memory usage: 15.8+ KB
